In [1]:
'''
Extract features from time series data of temperature.
- Statistical features in time domaion:(max, min, mean, std, var, median), (skew, kurt) for only 30, 60 minutes
- Time delta of Resampling : 10 minutes
- Window size of rolling window calculations : 10 minutes, 30 minutes, 1 hour(60 minutes)

Concetanate all above features (22 columns)
Construct labels(normal / abnormal_cold / abnormal_hot) with lag features for every hour 

'''
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import sys

## Read csv file of temperature data
df_Temp = pd.read_csv('ts_temperature.csv')
print (df_Temp.columns)
df_Temp['date'] = pd.to_datetime(df_Temp['date'], format="%Y-%m-%d %H:%M:%S")
# Set index as date
df_Temp = df_Temp.set_index('date')

print (df_Temp)
# print ("type(df_Temp['date'])", type(df_Temp['date']).__name__)
# print ("type(df_Temp['date'][0])", type(df_Temp['date'][0]).__name__)

Index(['date', 'temperature'], dtype='object')
                     temperature
date                            
2018-01-31 00:00:00    19.956961
2018-01-31 00:01:00    20.051956
2018-01-31 00:02:00    20.634282
2018-01-31 00:03:00    18.988631
2018-01-31 00:04:00    19.140233
...                          ...
2020-02-27 23:56:00    19.855776
2020-02-27 23:57:00    20.806417
2020-02-27 23:58:00    20.144426
2020-02-27 23:59:00    19.653403
2020-02-28 00:00:00    19.526193

[1091521 rows x 1 columns]


In [2]:
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 


## Feature calculation for window size 10
features = ['max', 'min', 'mean', 'std', 'var', 'median']
temprl = []

for f in features:
    print (f)
    temprl.append( df_Temp.resample('10T', closed='left', label='right', how=f) )

temp_10_min = pd.concat(temprl, axis=1)
temp_10_min.columns = [f + '_temp_10_min' for f in features]
temp_10_min.reset_index(inplace=True)

print ("temp_10_min", temp_10_min)
# temp_max_3mins=pd.DataFrame()
# temp_max_3mins = df_Temp.resample('10T', closed='left', label='right', how='min')
# temp_max_3mins.columns = ['10mins_'+'']
# temp_max_3mins.reset_index(inplace=True)
# print (type(temp_max_3mins))
# print ("temp_max_3mins", temp_max_3mins)


## Feature calculation for window size 30 and 60
features = ['max', 'min', 'mean', 'std', 'var', 'median', 'skew','kurt']
window_size = ['30T', '60T']
temprl = []
temp_rolling = []
for ws in window_size:
    print (ws)
    print (int(ws[:2]))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).max().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).min().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).mean().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).std().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).var().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).median().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).skew().resample('10T', closed='left', label='right',how='first'))
    temprl.append( df_Temp.rolling(window=int(ws[:2]),center=False).kurt().resample('10T', closed='left', label='right',how='first'))
    
    temp = pd.concat(temprl, axis=1)
    temp.columns = [f + '_temp_'+ str(ws[:2]) + '_min' for f in features]
    temp = temp.loc[-temp['max' + '_temp_'+ str(ws[:2]) + '_min'].isnull()]
    temp.reset_index(inplace=True)
    temp_rolling.append(temp)
    temprl = []
    
# temp_10_min = pd.concat(temprl, axis=1)
# temp_10_min.columns = [f + '_temp_10_min' for f in features]
# temp_10_min.reset_index(inplace=True)

# print ("temp_10_min", temp_10_min)
# print ("end of line")

print ("temp_rolling[0]", temp_rolling[0])
print ("temp_rolling[1]", temp_rolling[1])


temp_feat = pd.concat([temp_10_min, temp_rolling[0].ix[:, 1:9], temp_rolling[1].ix[:, 1:9]], axis=1).dropna()
print ("temp_feat", temp_feat)



max
min
mean
std
var
median
temp_10_min                       date  max_temp_10_min  min_temp_10_min  \
0      2018-01-31 00:10:00        20.723264        18.585998   
1      2018-01-31 00:20:00        20.785159        18.713738   
2      2018-01-31 00:30:00        20.190654        18.773051   
3      2018-01-31 00:40:00        20.208956        18.939912   
4      2018-01-31 00:50:00        20.083399        19.404427   
...                    ...              ...              ...   
109148 2020-02-27 23:30:00        20.921166        17.797888   
109149 2020-02-27 23:40:00        20.984767        19.217403   
109150 2020-02-27 23:50:00        20.639841        18.936471   
109151 2020-02-28 00:00:00        20.806417        18.987868   
109152 2020-02-28 00:10:00        19.526193        19.526193   

        mean_temp_10_min  std_temp_10_min  var_temp_10_min  median_temp_10_min  
0              19.661192         0.695999         0.484415           19.634942  
1              19.867153     

/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).max()
  # This is added back by InteractiveShellApp.init_path()
/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).min()
  # This is added back by InteractiveShellApp.init_path()
/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  # This is added back by InteractiveShellApp.init_path()
/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).std()
  # This is added back by InteractiveShellApp.init_path()
/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: how in .r

60T
60
temp_rolling[0]                       date  max_temp_30_min  min_temp_30_min  \
0      2018-01-31 00:30:00        20.785159        18.585998   
1      2018-01-31 00:40:00        20.785159        18.585998   
2      2018-01-31 00:50:00        20.785159        18.713738   
3      2018-01-31 01:00:00        20.208956        18.508526   
4      2018-01-31 01:10:00        20.208956        17.991682   
...                    ...              ...              ...   
109146 2020-02-27 23:30:00        21.102557        19.077929   
109147 2020-02-27 23:40:00        21.083886        17.797888   
109148 2020-02-27 23:50:00        20.984767        17.797888   
109149 2020-02-28 00:00:00        20.984767        17.797888   
109150 2020-02-28 00:10:00        20.984767        18.936471   

        mean_temp_30_min  std_temp_30_min  var_temp_30_min  \
0              19.662758         0.614867         0.378062   
1              19.656502         0.612722         0.375428   
2              19.7101

/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
/home/hyunhomo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


In [3]:
## Label construction
# labeled_features = final_feat.merge(failures, on=['datetime', 'machineID'], how='left')

# 1hr time delta resampling with min,max 
# Create indicator array ( min<17:cold, max>23:hot)
min_10_arr = temp_10_min['min_temp_10_min']
max_10_arr = temp_10_min['max_temp_10_min']
label_arr = np.zeros(len(max_10_arr), dtype = object)
# label_arr = np.empty(len(max_60_arr))
# label_arr[:] = np.nan
print ("label_arr", label_arr)


label_arr[((min_10_arr>=17) & (max_10_arr<=23)) ] = 'normal'
label_arr[min_10_arr<17] = 'abnormal(cold)'
label_arr[max_10_arr>23] ='abnormal(hot)'

print ("label_arr", label_arr)
print (np.sum([label_arr == 'normal']))
print (np.sum([label_arr == 'abnormal(cold)']))
print (np.sum([label_arr == 'abnormal(hot)']))

temp_10_min['label'] = label_arr
print ("temp_10_min", temp_10_min)
# comp_rep.loc[comp_rep[comp] < 1, comp] = None


# Concatenate to feature frame
# labeled_temp_feat = temp_feat.merge(label, on=['date'], how='left')
labeled_temp_feat = pd.merge(temp_feat,temp_10_min[['date','label']],on='date', how='left')
# Convert 'normal'state as 'NaN' (same as missing data aligned to date)
labeled_temp_feat.loc[labeled_temp_feat['label'] == 'normal','label'] = np.nan
pd.set_option('display.max_rows', 1000)
# Fill N/A with bfill limit 5 (fill backward up to 1hr (60mins))
labeled_temp_feat['label'] = labeled_temp_feat['label'].fillna(method='bfill', limit=5)
# Otherwise(normal operation) should be 'none'
labeled_temp_feat['label'] = labeled_temp_feat['label'].fillna('none')
# print ("labeled_temp_feat", labeled_temp_feat)
# print ("labeled_temp_feat.head(500)", labeled_temp_feat.head(500))



pd.set_option('display.max_rows', 1000)
print (labeled_temp_feat[['min_temp_10_min','max_temp_10_min','date','label']].head(1000))

labeled_temp_feat.to_csv('labeled_temp_feat.csv', index=False)
print ("csv file saved")

label_arr [0 0 0 ... 0 0 0]
label_arr ['normal' 'normal' 'normal' ... 'normal' 'normal' 'normal']
106696
1187
1270
temp_10_min                       date  max_temp_10_min  min_temp_10_min  \
0      2018-01-31 00:10:00        20.723264        18.585998   
1      2018-01-31 00:20:00        20.785159        18.713738   
2      2018-01-31 00:30:00        20.190654        18.773051   
3      2018-01-31 00:40:00        20.208956        18.939912   
4      2018-01-31 00:50:00        20.083399        19.404427   
...                    ...              ...              ...   
109148 2020-02-27 23:30:00        20.921166        17.797888   
109149 2020-02-27 23:40:00        20.984767        19.217403   
109150 2020-02-27 23:50:00        20.639841        18.936471   
109151 2020-02-28 00:00:00        20.806417        18.987868   
109152 2020-02-28 00:10:00        19.526193        19.526193   

        mean_temp_10_min  std_temp_10_min  var_temp_10_min  \
0              19.661192         0.695999 

csv file saved
